In [1]:
import torch
try:
    from transformers import AutoTokenizer, AutoModel
except:
    !pip install transformers
    from transformers import  AutoTokenizer,AutoModel

from sklearn.svm import SVC
import json
import pandas as pd
import pickle
import random
from tqdm import tqdm
import os 
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

2022-01-14 11:49:30.764435: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64
2022-01-14 11:49:30.764482: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
tokenizer = AutoTokenizer.from_pretrained('Davlan/bert-base-multilingual-cased-ner-hrl')
model = AutoModel.from_pretrained('Davlan/bert-base-multilingual-cased-ner-hrl',output_attentions=True).cuda()


Some weights of the model checkpoint at Davlan/bert-base-multilingual-cased-ner-hrl were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at Davlan/bert-base-multilingual-cased-ner-hrl and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
def encodings_context(sentence, keywords, tokenizer, model):
    tokens =  tokenizer.encode(sentence,max_length=512, truncation=True)

    extracted_keywords = []
    indices = []
    for keyword in keywords:
        keyword_tokens = tokenizer.encode(keyword)[1:-1]
        index = -1
        for i in range(len(tokens)):
            if tokens[i:i+len(keyword_tokens)] == keyword_tokens:
                index=i
                break
        if index >=0:
            indices.append((i,i+len(keyword_tokens)))
            extracted_keywords.append(keyword)
    out = model(torch.tensor([tokens]).cuda())
    embeddings = []
    for s, e in indices:     
        embeddings.append(out.last_hidden_state[0][s:e].mean(0).detach().cpu().numpy())
    return extracted_keywords, embeddings

In [4]:
def encoding_context(sentence, keyword, tokenizer, model):
    tokens = tokenizer.encode(sentence)
    keyword_tokens = tokenizer.encode(keyword)[1:-1]
    index = -1
    for i in range(len(tokens)):
        if tokens[i:i+len(keyword_tokens)] == keyword_tokens:
            index=i
            break
    assert index >=0, "Keyword '{}' not found in '{}'".format(keyword, sentence)
    out = model(torch.tensor([tokens]).cuda())
    return out.last_hidden_state[0][i:i+len(keyword_tokens)].mean(0).detach().cpu().numpy()

In [5]:
tech_embeddings_de = []
df = pd.read_csv('./domain_sentences/c4istar_technologies_sentences_de.csv')
for i in tqdm(range(len(df))):
    try:
        embedding = encoding_context(df['sentence'].iloc[i].lower(), df['kw'].iloc[i].lower(), tokenizer, model)
    except:
        pass
        # print(df['sentence'].iloc[i], [df['kw'].iloc[i]])
    else:
        tech_embeddings_de.append(embedding)
tech_embeddings_fr = []
df = pd.read_csv('./domain_sentences/c4istar_technologies_sentences_fr.csv')
for i in tqdm(range(len(df))):
    try:
        embedding = encoding_context(df['sentence'].iloc[i].lower(), df['kw'].iloc[i].lower(), tokenizer, model)
    except:
        pass
        # print(df['sentence'].iloc[i], [df['kw'].iloc[i]])
    else:
        tech_embeddings_fr.append(embedding)
eco_embeddings_de = []
df = pd.read_csv('./domain_sentences/eco_sentences_de.csv')
for i in tqdm(range(len(df))):
    try:
        embedding = encoding_context(df['sentence'].iloc[i].lower(), df['kw'].iloc[i].lower(), tokenizer, model)
    except:
        pass
    else:
        eco_embeddings_de.append(embedding)
eco_embeddings_fr = []
df = pd.read_csv('./domain_sentences/eco_sentences_fr.csv')
for i in tqdm(range(len(df))):
    try:
        embedding = encoding_context(df['sentence'].iloc[i].lower(), df['kw'].iloc[i].lower(), tokenizer, model)
    except:
        pass
        # print(df['sentence'].iloc[i], [df['kw'].iloc[i]])
    else:
        eco_embeddings_fr.append(embedding)
tech_embeddings = []
for kw, sentences  in tqdm(json.load(open('domain_sentences/c4istar_technologies_sentences.txt')).items()):
    for sentence in sentences:
        try:
            tech_embeddings.append(encoding_context(sentence.lower(), kw.lower(), tokenizer, model))
        except:
            pass
            # print(kw, sentence)
eco_embeddings = []
for kw, sentences  in tqdm(json.load(open('domain_sentences/eco_sentences.json')).items()):
    for sentence in sentences:
        try:
            eco_embeddings.append(encoding_context(sentence.lower(), kw.lower(), tokenizer, model))
        except Exception as e:
            pass
            # print(e, kw, sentence)   

100%|█████████████████████████████████████████| 229/229 [08:15<00:00,  2.17s/it]


In [6]:
random.shuffle(tech_embeddings)
random.shuffle(tech_embeddings_de)
random.shuffle(tech_embeddings_fr)
random.shuffle(eco_embeddings)
random.shuffle(eco_embeddings_de)
random.shuffle(eco_embeddings_fr)

In [7]:
clf = SVC(gamma='auto',probability=True)
clf.fit(tech_embeddings+eco_embeddings[:len(tech_embeddings)], [0]*len(tech_embeddings)+[1]*len(tech_embeddings))
clf.score(tech_embeddings_fr+tech_embeddings_de+eco_embeddings_fr+eco_embeddings_de, [0]*len(tech_embeddings_fr+tech_embeddings_de)+[1]*len(eco_embeddings_fr+eco_embeddings_de))

0.7971811619113097

In [8]:
pickle.dump(clf, open('svm.pkl','wb'))
